In [1]:
from functions import *

In [2]:
# Read the CSV file into a DataFrame
result_dir = "../../exp_results"

In [3]:
summary_path = result_dir + "/result_summary.csv"
output_path = result_dir + "/correctness_per_dataset.csv"

check_correntness(summary_path, output_path)

    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.2  50   0   0   0      1.00
1        3.0          0.4  45   3   0   2      0.96
2        3.0          0.6  46   4   0   0      1.00
3        3.0          0.8  31  18   0   1      0.98
4        6.0          0.2  48   0   0   2      0.96
5        6.0          0.4  37   9   0   4      0.92
6        6.0          0.6  32  13   0   5      0.90
7        6.0          0.8  16  31   0   3      0.94
8        9.0          0.2  44   0   0   6      0.88
9        9.0          0.4  33   9   0   8      0.84
10       9.0          0.6  20  24   0   6      0.88
11       9.0          0.8   6  41   0   3      0.94
12      12.0          0.2  41   0   0   9      0.82
13      12.0          0.4  21  15   0  14      0.72
14      12.0          0.6  14  26   0  10      0.80
15      12.0          0.8   4  43   0   3      0.94
16      15.0          0.2  36   0   0  14      0.72
17      15.0          0.4  15  19   0  16      0.68
18      15.0

In [4]:
summary_path = result_dir + "/result_summary.csv"

num_cores = 1
num_tasks = 3
utilization =0.4 
detail_result_dir = result_dir + "/detail_result"

FN_subset_info = search_FN_info(summary_path, num_tasks, utilization)
print("FN_info")
print(FN_subset_info)

taskset_indice = FN_subset_info["tasksetIndex"].tolist()
for taskset_index in taskset_indice:
    print("\n\nTaskset_index: ", taskset_index)
    detail_result = show_detail_result(detail_result_dir, num_cores, num_tasks, utilization, taskset_index)
    print(detail_result)

FN_info
    nTasks  util  tasksetIndex  sched(Sim)  sched(proposed)
52       3   0.4             2        True            False
70       3   0.4            20        True            False


Taskset_index:  2
    WCRT(Sim)  sched(Sim)  WCRT(proposed)  sched(proposed)
id                                                        
1       74861        True           77469             True
2      132742        True          132742             True
3       92316        True          100571            False


Taskset_index:  20
    WCRT(Sim)  sched(Sim)  WCRT(proposed)  sched(proposed)
id                                                        
1      316848        True          317124             True
2      147210        True          168275             True
3      185340        True          201336            False


In [5]:
# check conservativeness of results for each task
detail_result_dir = result_dir + "/detail_result"
combine_detail_result_path = detail_result_dir + "/combined_detail_result.csv"
combine_detail_result(summary_path,  detail_result_dir, combine_detail_result_path)

output_path = result_dir + "/correctness_per_task.csv"
check_correntness(combine_detail_result_path, output_path)

    numTasks  utilization   TP  TN  FP  FN  accuracy
0        3.0          0.2  150   0   0   0  1.000000
1        3.0          0.4  139   0   0   2  0.985816
2        3.0          0.6  138   0   0   0  1.000000
3        3.0          0.8   95   0   0   1  0.989583
4        6.0          0.2  298   0   0   2  0.993333
5        6.0          0.4  242   0   0   4  0.983740
6        6.0          0.6  217   0   0   5  0.977477
7        6.0          0.8  111   0   0   3  0.973684
8        9.0          0.2  444   0   0   6  0.986667
9        9.0          0.4  361   0   0   8  0.978320
10       9.0          0.6  227   0   0   7  0.970085
11       9.0          0.8   77   0   0   4  0.950617
12      12.0          0.2  590   0   0  10  0.983333
13      12.0          0.4  406   0   0  14  0.966667
14      12.0          0.6  277   1   0  10  0.965278
15      12.0          0.8   77   0   0   7  0.916667
16      15.0          0.2  733   1   0  16  0.978667
17      15.0          0.4  446   0   0  19  0.

In [6]:
import pandas as pd

combine_detail_result = pd.read_csv(combine_detail_result_path, sep=",")
TN_subset_info = combine_detail_result[combine_detail_result["simulator_schedulability"] == False]
print(TN_subset_info)

      numCores  numTasks  utilization  tasksetIndex   name  WCRT(sim)   
3783         1        12          0.6            42  task7     103604  \
4069         1        15          0.2             9  task2     100330   

      simulator_schedulability  WCRT(prop)  proposed_schedulability  
3783                     False      162309                    False  
4069                     False      111996                    False  


In [7]:
## print results that WCRT(sim) > WCRT(proposed)
# print_non_conservative_results(combine_detail_result_path)

# assert that non convervative results are occured only when the system is not schedulable
# check_non_conservative_results(summary_path, combine_detail_result_path)